In [ ]:
%load_ext autoreload
%autoreload 2

import geoviews as gv
import geoviews.feature as gf
from cartopy import crs as ccrs

import paleoviews as pv
from gprm.datasets import Reconstructions

gv.extension('bokeh', 'matplotlib')



In [ ]:
M2021 = Reconstructions.fetch_Merdith2021()


In [ ]:
Continents = M2021.polygon_snapshot('continents', 250)


In [ ]:
dim = 300
central_longitude = 0.

gv.Polygons(pv.wrap_reconstructed_features(Continents.reconstructed_polygons, central_longitude=central_longitude), 
            vdims=['NAME', 'PLATEID1', 'FROMAGE']).opts(cmap='inferno_r',
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude), color_index='FROMAGE'
)

In [ ]:
Plates = M2021.plate_snapshot(250.)

central_longitude = 90.

features = [resolved_topology.get_resolved_feature() for resolved_topology in Plates.resolved_topologies]
dim = 300
gv.Polygons(pv.wrap_features(features), 
            vdims=['NAME', 'PLATEID1']).opts(
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude), fill_color='red'
)

In [ ]:
Plates = M2021.plate_snapshot(150.)
Continents = M2021.polygon_snapshot('continents', 150)
features = [resolved_topology.get_resolved_feature() for resolved_topology in Plates.resolved_topologies]

dim = 600
central_longitude = 0.


gv.Polygons(pv.wrap_features(features), 
            vdims=['NAME', 'PLATEID1']).opts(
    height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude), fill_color='red'
) * gv.Polygons(pv.wrap_reconstructed_features(Continents.reconstructed_polygons, central_longitude=central_longitude), 
            vdims=['NAME', 'PLATEID1', 'FROMAGE']).opts(cmap='inferno_r',
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude), color_index='FROMAGE'
)


In [ ]:
Plates = M2021.plate_snapshot(250.)

central_longitude = 90.

#features = [resolved_topology.get_resolved_feature() for resolved_topology in Plates.resolved_topological_sections]
features = Plates.get_boundary_features()
dim = 300
gv.Path(pv.wrap_features(features), 
            vdims=['NAME', 'PLATEID1']).opts(
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude),
)
